In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
import os


import json
import requests


# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)


from scipy.stats import linregress
#pip install citipy
# Import API key   

from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
#output_data_file = "output_data/cities.csv"



In [8]:
cityinfo = pd.read_csv("data/uscities-airport-lat-lng.csv")
cityinfo.head(10)

,city,state_id,state_name,lat,lng,population,airport name,airport id
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW
1,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago O'hare Airport,ORD
2,Las Vegas,NV,Nevada,36.2333,-115.2654,2165405,Las Vegas Airport,HND
3,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX
4,Miami,FL,Florida,25.7840,-80.2101,6076316,Miami Airport,MIA
5,New York,NY,New York,40.6943,-73.9249,18680025,John F. Kenned Airport,JFK
6,New York,NY,New York,40.6943,-73.9249,18680025,La Guardia Airport,LGA
7,Orlando,FL,Florida,28.4773,-81.3370,1792047,Orlando Airport,MCO
8,San Francisco,CA,California,37.7558,-122.4449,3557982,San Francisco Airport,SFO
9,Seattle,WA,Washington,47.6211,-122.3244,3530752,Seattle Airport,SEA


In [9]:
cities = cityinfo['city']


In [10]:
population = cityinfo['population']

In [11]:
cities

0           Chicago
1           Chicago
2         Las Vegas
3       Los Angeles
4             Miami
5          New York
6          New York
7           Orlando
8     San Francisco
9           Seattle
10       Washington
11       Washington
12           Boston
Name: city, dtype: object

In [12]:
population

0      8586888
1      8586888
2      2165405
3     12531334
4      6076316
5     18680025
6     18680025
7      1792047
8      3557982
9      3530752
10     5066973
11     5066973
12     4465966
Name: population, dtype: int64

In [13]:
len(cities)

13

In [14]:
#  https://api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}

#url = "http://api.openweathermap.org/data/2.5/weather?"
url = "https://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

#for city in cities:
# Build query URL
query_url = url + "appid=" + weather_api_key + "&units=" + units +  "&q=" + cities[0]

#query_url

# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()

# Get the temperature from the response
#print(f"The weather API responded with: {weather_json}.")

y = json.dumps(weather_json, indent=4, separators=(". ", " = "))
print(y)

{
    "coord" = {
        "lon" = -87.65. 
        "lat" = 41.85
    }. 
    "weather" = [
        {
            "id" = 800. 
            "main" = "Clear". 
            "description" = "clear sky". 
            "icon" = "01d"
        }
    ]. 
    "base" = "stations". 
    "main" = {
        "temp" = 80.73. 
        "feels_like" = 82.31. 
        "temp_min" = 77.72. 
        "temp_max" = 82.6. 
        "pressure" = 1009. 
        "humidity" = 57
    }. 
    "visibility" = 10000. 
    "wind" = {
        "speed" = 9.22. 
        "deg" = 350
    }. 
    "clouds" = {
        "all" = 0
    }. 
    "dt" = 1658157776. 
    "sys" = {
        "type" = 2. 
        "id" = 2005153. 
        "country" = "US". 
        "sunrise" = 1658140268. 
        "sunset" = 1658193728
    }. 
    "timezone" = -18000. 
    "id" = 4887398. 
    "name" = "Chicago". 
    "cod" = 200
}


In [15]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

#for city in cities:
# Build query URL
query_url = url + "appid=" + weather_api_key + "&units=" + units +  "&q=" 


city_name = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []

#set a counter
index_counter = 0
set_counter = 1


print("start Data Retrieval")
print("-"*10)

for index, city in enumerate (cities):
    
        response = requests.get(query_url + city).json()
        try:
            city_name.append(response['name'])
            lat.append(response['coord']['lat'])
            lng.append(response['coord']['lon'])
            max_temp.append(response['main']['temp_max'])
            humidity.append(response['main']['humidity'])
            cloudiness.append(response['clouds']['all'])
            wind_speed.append(response['wind']['speed'])
            country.append(response['sys']['country'])
            date.append(response['dt'])
           
            
            if index_counter > 49:
                index_counter = 0
                set_counter = set_counter + 1
    
            else:
                index_counter = index_counter + 1
            
                print(f"Processing Record {index_counter} of Set {set_counter} : {city}") 
  
        except(KeyError, IndexError):
            print("City not found...")

print("-"*10)
print("Data Retrieval Complete")
print("-"*10)

start Data Retrieval
----------
Processing Record 1 of Set 1 : Chicago
Processing Record 2 of Set 1 : Chicago
Processing Record 3 of Set 1 : Las Vegas
Processing Record 4 of Set 1 : Los Angeles
Processing Record 5 of Set 1 : Miami
Processing Record 6 of Set 1 : New York
Processing Record 7 of Set 1 : New York
Processing Record 8 of Set 1 : Orlando
Processing Record 9 of Set 1 : San Francisco
Processing Record 10 of Set 1 : Seattle
Processing Record 11 of Set 1 : Washington
Processing Record 12 of Set 1 : Washington
Processing Record 13 of Set 1 : Boston
----------
Data Retrieval Complete
----------


In [16]:
city_weather_1 = pd.DataFrame({'city': city_name,
                            
                             'Max Temp': max_temp,
                             'Humidity': humidity, 
                             'Cloudiness': cloudiness,
                             'Wind Speed': wind_speed,
                             'Country': country,
                             'Date': date
                             
                            })
city_weather_1.head()

,city,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,82.60,57,0,9.22,US,1658157776
1,Chicago,82.60,57,0,9.22,US,1658157776
2,Las Vegas,93.58,32,0,9.22,US,1658157764
3,Los Angeles,83.44,74,0,11.50,US,1658158029
4,Miami,92.01,73,75,14.97,US,1658157661


In [17]:
city_weather_1['Date'] = pd.to_datetime(city_weather_1['Date'], unit='s')
city_weather_1['Date'] = city_weather_1['Date'].dt.strftime('%Y-%m-%d')

In [18]:
city_weather_1

,city,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,82.60,57,0,9.22,US,2022-07-18
1,Chicago,82.60,57,0,9.22,US,2022-07-18
2,Las Vegas,93.58,32,0,9.22,US,2022-07-18
3,Los Angeles,83.44,74,0,11.50,US,2022-07-18
4,Miami,92.01,73,75,14.97,US,2022-07-18
5,New York,83.10,83,75,9.22,US,2022-07-18
6,New York,83.10,83,75,9.22,US,2022-07-18
7,Orlando,90.95,65,0,13.80,US,2022-07-18
8,San Francisco,71.17,82,20,10.36,US,2022-07-18
9,Seattle,58.60,87,75,7.00,US,2022-07-18


In [19]:

city_weather_1.drop_duplicates(subset=['city'])

,city,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,82.60,57,0,9.22,US,2022-07-18
2,Las Vegas,93.58,32,0,9.22,US,2022-07-18
3,Los Angeles,83.44,74,0,11.50,US,2022-07-18
4,Miami,92.01,73,75,14.97,US,2022-07-18
5,New York,83.10,83,75,9.22,US,2022-07-18
7,Orlando,90.95,65,0,13.80,US,2022-07-18
8,San Francisco,71.17,82,20,10.36,US,2022-07-18
9,Seattle,58.60,87,75,7.00,US,2022-07-18
10,Washington,65.89,60,2,6.20,US,2022-07-18
12,Boston,83.07,74,100,9.22,US,2022-07-18


In [20]:
city_weather_1.to_csv("data/cities.csv")

In [21]:

city_weather = pd.merge(cityinfo, city_weather_1, how="left", on=["city", "city"])

In [22]:
city_weather

,city,state_id,state_name,lat,lng,population,airport name,airport id,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18
1,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18
2,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago O'hare Airport,ORD,82.60,57,0,9.22,US,2022-07-18
3,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago O'hare Airport,ORD,82.60,57,0,9.22,US,2022-07-18
4,Las Vegas,NV,Nevada,36.2333,-115.2654,2165405,Las Vegas Airport,HND,93.58,32,0,9.22,US,2022-07-18
5,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX,83.44,74,0,11.50,US,2022-07-18
6,Miami,FL,Florida,25.7840,-80.2101,6076316,Miami Airport,MIA,92.01,73,75,14.97,US,2022-07-18
7,New York,NY,New York,40.6943,-73.9249,18680025,John F. Kenned Airport,JFK,83.10,83,75,9.22,US,2022-07-18
8,New York,NY,New York,40.6943,-73.9249,18680025,John F. Kenned Airport,JFK,83.10,83,75,9.22,US,2022-07-18
9,New York,NY,New York,40.6943,-73.9249,18680025,La Guardia Airport,LGA,83.10,83,75,9.22,US,2022-07-18


In [23]:
city_weather = city_weather.drop_duplicates(subset=['city'])

In [24]:
city_weather


,city,state_id,state_name,lat,lng,population,airport name,airport id,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18
4,Las Vegas,NV,Nevada,36.2333,-115.2654,2165405,Las Vegas Airport,HND,93.58,32,0,9.22,US,2022-07-18
5,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX,83.44,74,0,11.50,US,2022-07-18
6,Miami,FL,Florida,25.7840,-80.2101,6076316,Miami Airport,MIA,92.01,73,75,14.97,US,2022-07-18
7,New York,NY,New York,40.6943,-73.9249,18680025,John F. Kenned Airport,JFK,83.10,83,75,9.22,US,2022-07-18
11,Orlando,FL,Florida,28.4773,-81.3370,1792047,Orlando Airport,MCO,90.95,65,0,13.80,US,2022-07-18
12,San Francisco,CA,California,37.7558,-122.4449,3557982,San Francisco Airport,SFO,71.17,82,20,10.36,US,2022-07-18
13,Seattle,WA,Washington,47.6211,-122.3244,3530752,Seattle Airport,SEA,58.60,87,75,7.00,US,2022-07-18
14,Washington,DC,District of Columbia,38.9047,-77.0163,5066973,Washington Regan Airport,DCA,65.89,60,2,6.20,US,2022-07-18
18,Boston,MA,Massachusetts,42.3188,-71.0852,4465966,Boston Airport,BOS,83.07,74,100,9.22,US,2022-07-18


In [25]:
city_weather.to_csv("Clean_data/city_weather.csv")

In [26]:
gmaps.configure(api_key=g_key)
locations = city_weather[["lat", "lng"]]
humidity = city_weather["Humidity"]
locations

,lat,lng
0,41.8375,-87.6866
4,36.2333,-115.2654
5,34.1141,-118.4068
6,25.7840,-80.2101
7,40.6943,-73.9249
11,28.4773,-81.3370
12,37.7558,-122.4449
13,47.6211,-122.3244
14,38.9047,-77.0163
18,42.3188,-71.0852


In [27]:
locations.dropna()

,lat,lng
0,41.8375,-87.6866
4,36.2333,-115.2654
5,34.1141,-118.4068
6,25.7840,-80.2101
7,40.6943,-73.9249
11,28.4773,-81.3370
12,37.7558,-122.4449
13,47.6211,-122.3244
14,38.9047,-77.0163
18,42.3188,-71.0852


In [28]:
humidity

0     57
4     32
5     74
6     73
7     83
11    65
12    82
13    87
14    60
18    74
Name: Humidity, dtype: int64

In [29]:
fit_weather_data = city_weather.loc[(city_weather['Wind Speed']<= 20) & (city_weather['Cloudiness'] == 0) & 
                                    (city_weather['Max Temp']<=90) & (city_weather['Max Temp'] >= 70)]
fit_weather_data.dropna()
fit_weather_data.head()

,city,state_id,state_name,lat,lng,population,airport name,airport id,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18
5,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX,83.44,74,0,11.50,US,2022-07-18


In [30]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
hotel_df = fit_weather_data.loc[:,:]
hotel_df['Hotel Name'] = ""

target_type = "hotel"
radius = 5000

# rewrite params dict

params = {
    "keyword" : "hotel",
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    city_name = row["city"]

    params["location"] = f"{row['lat']}, {row['lng']}"
    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
   #  time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

C:\Users\vasan\AppData\Local\Temp/ipykernel_18816/2246913812.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


Retrieving Results for Index 0: Chicago.
Closest hotel in Chicago is Hotel Chicago West Loop.
Retrieving Results for Index 5: Los Angeles.


c:\Users\vasan\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel in Los Angeles is Beverly Laurel Motor Hotel.
-------End of Search-------


In [32]:
hotel_df

,city,state_id,state_name,lat,lng,population,airport name,airport id,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18,Hotel Chicago West Loop
5,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX,83.44,74,0,11.50,US,2022-07-18,Beverly Laurel Motor Hotel


In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [34]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(height='420px'))

### Database connection

In [40]:
import psycopg2
from sqlalchemy import create_engine
import config

In [41]:
# Connect to your Postgres database
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(config.username, config.password, 'localhost', 5432, 'city_weather_db'))
con = engine.connect()


In [44]:
# check for tablenames in the database
engine.table_names()

C:\Users\vasan\AppData\Local\Temp/ipykernel_18816/3623636692.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['city_weather', 'fit_weather_data', 'hotel']

In [45]:
# use pandas to load  city_weather dataframe into the database
city_weather.to_sql('city_weather', con, if_exists='replace', index=False)

In [48]:
# confirm that the data was loaded into the city_weather table
pd.read_sql_query('select * from city_weather', con=engine).head()

,city,state_id,state_name,lat,lng,population,airport name,airport id,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18
1,Las Vegas,NV,Nevada,36.2333,-115.2654,2165405,Las Vegas Airport,HND,93.58,32,0,9.22,US,2022-07-18
2,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX,83.44,74,0,11.50,US,2022-07-18
3,Miami,FL,Florida,25.7840,-80.2101,6076316,Miami Airport,MIA,92.01,73,75,14.97,US,2022-07-18
4,New York,NY,New York,40.6943,-73.9249,18680025,John F. Kenned Airport,JFK,83.10,83,75,9.22,US,2022-07-18


In [46]:
# use pandas to load  fit_weather_data dataframe into the database
fit_weather_data.to_sql('fit_weather_data', con, if_exists='replace', index=False)

In [49]:
# confirm that the data was loaded into the fit_weather_data table
pd.read_sql_query('select * from fit_weather_data', con=engine).head()

,city,state_id,state_name,lat,lng,population,airport name,airport id,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18,Hotel Chicago West Loop
1,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX,83.44,74,0,11.50,US,2022-07-18,Beverly Laurel Motor Hotel


In [47]:
# use pandas to load  hotel dataframe into the database
hotel_df.to_sql('hotel', con, if_exists='replace', index=False)

In [50]:
# confirm that the data was loaded into the hotel table
pd.read_sql_query('select * from hotel', con=engine).head()

,city,state_id,state_name,lat,lng,population,airport name,airport id,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Chicago,IL,Illinois,41.8375,-87.6866,8586888,Chicago Midway Airport,MDW,82.60,57,0,9.22,US,2022-07-18,Hotel Chicago West Loop
1,Los Angeles,CA,California,34.1141,-118.4068,12531334,Los Angeles Airport,LAX,83.44,74,0,11.50,US,2022-07-18,Beverly Laurel Motor Hotel
